In [1]:
import time
import re
import requests as rt
import bs4
import json
import warnings
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
warnings.filterwarnings("ignore")
import pickle
import numpy
import threading
import numpy as np
import sqlite3 as sq
from collections import deque
from tqdm import tqdm

In [2]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [3]:
def value_tester(index):
    print(product_links[index])
    print(mrp[index])
    print(price[index])
    print(description[index])
    print(gender[index])
    print(brand_names[index])
    print(product_names[index])
    print(category[index])
    print(primary_image_links_dict[index])
    print(secondary_image_links_dict[index])
    print(sizes[index])
    

In [4]:
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}

# Women


## Tops

In [5]:
product_links={}
counter=0
for i in range(1,7):
    data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fwomen%2Ftops&page-size=100&sort=relevance&page={i}',headers=headers)
    data_in_json=json.loads(data.text)
    for j in data_in_json['data'][0]['data']:
        product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
        counter+=1
    

In [6]:
len(product_links)

600

In [7]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
category={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Tops'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break

In [8]:
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(category)

(600, 599, 599, 599, 599, 599, 599, 599, 599, 599)

In [9]:
value_tester(555)

https://www.koovs.com/blue-saint-womens-tops-150204-167422.html
1399
1399
Made from Cotton V neck Full sleeves Bell sleeves Regular fit
Women
Blue Saint
Bell Sleeves Casual Top
Tops
https://product.koovs.com/167422_d13e7e4daab94dd3bbc00140bf0e4165_image1_super_zoom.jpg
['https://product.koovs.com/167422_d13e7e4daab94dd3bbc00140bf0e4165_front_super_zoom.jpg', 'https://product.koovs.com/167422_d13e7e4daab94dd3bbc00140bf0e4165_back_super_zoom.jpg', 'https://product.koovs.com/167422_d13e7e4daab94dd3bbc00140bf0e4165_left_super_zoom.jpg', 'https://product.koovs.com/167422_d13e7e4daab94dd3bbc00140bf0e4165_right_super_zoom.jpg']
['S', 'XL']


In [10]:
con=sq.connect("Koovs.db")  
cur=con.cursor()
cur.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

## Dresses

In [11]:
product_links={}
counter=0
for i in range(1,7):
    data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fwomen%2Fdresses&page-size=36&sort=relevance&page={i}',headers=headers)
    data_in_json=json.loads(data.text)
    for j in data_in_json['data'][0]['data']:
        product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
        counter+=1
    

In [12]:
len(product_links)

216

In [13]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
category={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Dresses'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break

In [14]:
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(category)

(216, 216, 216, 216, 216, 216, 216, 216, 216, 216)

In [15]:
value_tester(200)

https://www.koovs.com/blue-saint-womens-dresses-140174-157231.html
1499
899
Made from cotton-elastane blend Spread collar Button-down closure All over multi-vertical stripes Twin patch pocket Sleeveless Regular fit
Women
Blue Saint
Multi Vertical Stripe Shirt Dress
Dresses
https://product.koovs.com/157231_4908b934432948fd88b5e3ccd3589d67_image1_super_zoom.jpg
['https://product.koovs.com/157231_4908b934432948fd88b5e3ccd3589d67_front_super_zoom.jpg', 'https://product.koovs.com/157231_4908b934432948fd88b5e3ccd3589d67_back_super_zoom.jpg', 'https://product.koovs.com/157231_4908b934432948fd88b5e3ccd3589d67_left_super_zoom.jpg']
['XS', 'S', 'M', 'L']


In [16]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

## Jeans

In [17]:
product_links={}
counter=0
for i in range(1,7):
    try:
        data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fwomen%2Fjeans&page-size=36&sort=relevance&page={i}',headers=headers)
        data_in_json=json.loads(data.text)
        for j in data_in_json['data'][0]['data']:
            product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
            counter+=1
    except:
        pass

    

In [18]:
len(product_links)

153

In [19]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
category={}
mrp={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Jeans-Women'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break

In [20]:
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(category)

(153, 153, 153, 153, 153, 153, 153, 153, 153, 153)

In [21]:
value_tester(100)

https://www.koovs.com/mid-blue-eco-washed-boyfriend-jeans-151314-168536.html
1899
1234
Made from cotton-elastane Button and fly closure Classic five-pocket style Belt loops Straight fit
Women
Freakins
Mid Rise Boyfriend Jeans
Jeans-Women
https://product.koovs.com/168536_79f948137c764459a2e77a7cbb7b5b3f_image1_super_zoom.jpg
['https://product.koovs.com/168536_79f948137c764459a2e77a7cbb7b5b3f_front_super_zoom.jpg', 'https://product.koovs.com/168536_79f948137c764459a2e77a7cbb7b5b3f_back_super_zoom.jpg', 'https://product.koovs.com/168536_79f948137c764459a2e77a7cbb7b5b3f_left_super_zoom.jpg', 'https://product.koovs.com/168536_79f948137c764459a2e77a7cbb7b5b3f_right_super_zoom.jpg']
['30']


In [22]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

## Skirts

In [23]:
product_links={}
counter=0
for i in range(0,7):
    try:
        data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fwomen%2Fskirts&page-size=36&sort=relevance&page={0}',headers=headers)
        data_in_json=json.loads(data.text)
        for j in data_in_json['data'][0]['data']:
            product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
            counter+=1
    except:
        continue

In [24]:
len(product_links)

252

In [25]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
category={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Skirts'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break

In [26]:
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(category)

(252, 252, 252, 252, 252, 252, 252, 252, 252, 252)

In [27]:
value_tester(200)

https://www.koovs.com/dbasic-women-mini-skirts-151735-168957.html
999
999
Made from cotton Elasticated waist Button down style Mini length Regular fit
Women
D'BASIC
Button Down Betty Skirts
Skirts
https://product.koovs.com/168957_6d115161f49d4f28875cf33b327ed45a_image1_super_zoom.jpg
['https://product.koovs.com/168957_6d115161f49d4f28875cf33b327ed45a_front_super_zoom.jpg', 'https://product.koovs.com/168957_6d115161f49d4f28875cf33b327ed45a_back_super_zoom.jpg', 'https://product.koovs.com/168957_6d115161f49d4f28875cf33b327ed45a_left_super_zoom.jpg']
['S', 'M']


In [28]:
len(product_links)

252

In [29]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

## Leggings

In [30]:
product_links={}
counter=0
for i in range(0,7):
    try:
        data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fwomen%2Ftrousers-and-leggings%2Fsortby-latest&page-size=36&sort=relevance&page={i}',headers=headers)
        data_in_json=json.loads(data.text)
        for j in data_in_json['data'][0]['data']:
            product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
            counter+=1
    except:
        continue

In [31]:
len(product_links)

108

In [32]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
category={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Leggings'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break

In [33]:
value_tester(50)

https://www.koovs.com/koovs-womens-regular-jeans-151950-169172.html
1999
1999
Made from cotton-spandex blend Elastictaed waist Pleated style Four pockets Slim fit
Women
KOOVS
Elasticated Waist Pleated Trousers
Leggings
https://product.koovs.com/169172_fd82de4355b34c4aa062c6ca4d40d203_image1_super_zoom.jpg
['https://product.koovs.com/169172_fd82de4355b34c4aa062c6ca4d40d203_front_super_zoom.jpg', 'https://product.koovs.com/169172_fd82de4355b34c4aa062c6ca4d40d203_back_super_zoom.jpg', 'https://product.koovs.com/169172_fd82de4355b34c4aa062c6ca4d40d203_left_super_zoom.jpg', 'https://product.koovs.com/169172_fd82de4355b34c4aa062c6ca4d40d203_right_super_zoom.jpg']
['26', '28', '30', '32', '34']


In [34]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

## Shorts

In [35]:
product_links={}
counter=0
for i in range(0,7):
    try:
        data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fwomen%2Fshorts&page-size=36&sort=latest&page={i}',headers=headers)
        data_in_json=json.loads(data.text)
        for j in data_in_json['data'][0]['data']:
            product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
            counter+=1
    except:
        continue

In [36]:
len(product_links)

67

In [37]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
category={}
mrp={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Shorts'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break

In [38]:
value_tester(50)

https://www.koovs.com/onezero-womens-day-shorts-148732-165945.html
999
699
Made from poly-elastane blend Elasticated waist Textured effect Active stretch Breathable and lightweight Moisture-wicking technology Reflective brand logo Bodycon fit
Women
ONE/ZERO BY KOOVS
Active Stretch Training Shorts
Shorts
https://product.koovs.com/165945_77fd32bf7e1d4ad5ac3054d569779350_image1_super_zoom.jpg
['https://product.koovs.com/165945_77fd32bf7e1d4ad5ac3054d569779350_front_super_zoom.jpg', 'https://product.koovs.com/165945_77fd32bf7e1d4ad5ac3054d569779350_back_super_zoom.jpg', 'https://product.koovs.com/165945_77fd32bf7e1d4ad5ac3054d569779350_left_super_zoom.jpg', 'https://product.koovs.com/165945_77fd32bf7e1d4ad5ac3054d569779350_right_super_zoom.jpg']
['S', 'M', 'L', 'XL']


In [39]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

## Coats/Jackets

In [40]:
product_links={}
counter=0
for i in range(0,7):
    try:
        data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fwomen%2Fcoats-and-jackets&page-size=36&sort=relevance&page={i}',headers=headers)
        data_in_json=json.loads(data.text)
        for j in data_in_json['data'][0]['data']:
            product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
            counter+=1
    except:
        continue
len(product_links)

117

In [41]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
category={}
mrp={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Coats/Jackets'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

(117, 117, 117, 117, 117, 117, 117, 117, 117)

In [42]:
value_tester(50)

https://www.koovs.com/blue-graphic-women-quilted-jacket-144336-161442.html
2899
1450
Made From Polyester Hoodie Neckline Long Sleeves Floral Print Style Regular Fit
Women
Oxolloxo
Floral Print Jacket
Coats/Jackets
https://product.koovs.com/161442_7aba4754f433498f810156dbf5d05eb9_image1_super_zoom.jpg
['https://product.koovs.com/161442_7aba4754f433498f810156dbf5d05eb9_front_super_zoom.jpg', 'https://product.koovs.com/161442_7aba4754f433498f810156dbf5d05eb9_back_super_zoom.jpg', 'https://product.koovs.com/161442_7aba4754f433498f810156dbf5d05eb9_left_super_zoom.jpg']
['S', 'M', 'L', 'XL']


In [43]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

## Shoes

In [44]:
product_links={}
counter=0
for i in range(0,7):
    try:
        data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fwomen%2Fshoes&page-size=36&sort=relevance&page={i}',headers=headers)
        data_in_json=json.loads(data.text)
        for j in data_in_json['data'][0]['data']:
            product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
            counter+=1
    except:
        continue
len(product_links)

43

In [45]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
category={}
mrp={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Shoes'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

(43, 43, 43, 43, 43, 43, 43, 43, 43)

In [46]:
value_tester(40)

https://www.koovs.com/thong-flat-black-141316-158380.html
999
500
Made from PU Thong T-strap Metallic embellishment Flat sole
Women
INARI
Thong T-Strap Flat Sandals
Shoes
https://product.koovs.com/158380_e2bf16a4bf2749dd8d6108e41fcd1683_image1_super_zoom.jpg
['https://product.koovs.com/158380_e2bf16a4bf2749dd8d6108e41fcd1683_front_super_zoom.jpg', 'https://product.koovs.com/158380_e2bf16a4bf2749dd8d6108e41fcd1683_back_super_zoom.jpg', 'https://product.koovs.com/158380_e2bf16a4bf2749dd8d6108e41fcd1683_left_super_zoom.jpg']
['UK 5/EU 38', 'UK 6/EU 39', 'UK 7/EU 40', 'UK 4/EU 37']


In [47]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

# Men

## Shirts

In [48]:
product_links={}
counter=0
for i in range(0,7):
    try:
        data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fmen%2Fshirts&page-size=100&sort=relevance&page={i}',headers=headers)
        data_in_json=json.loads(data.text)
        for j in data_in_json['data'][0]['data']:
            product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
            counter+=1
    except:
        continue
len(product_links)

625

In [49]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
category={}
mrp={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Shirts'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

(625, 625, 625, 625, 625, 625, 625, 625, 625)

In [50]:
value_tester(40)

https://www.koovs.com/blue-saint-men-casual-shirts-153023-170257.html
1899
1899
Made from cotton  Spread collar  Buttondown closure  Overlap placket  Twin patch pocket Twin flap pockets Long sleeves Regular fit 
Men
Blue Saint
Twin Patch Pocket Regular Fit Shirt
Shirts
https://product.koovs.com/170257_c00be7a2068d4d889e5b94fab24271c4_image1_super_zoom.jpg
['https://product.koovs.com/170257_c00be7a2068d4d889e5b94fab24271c4_front_super_zoom.jpg', 'https://product.koovs.com/170257_c00be7a2068d4d889e5b94fab24271c4_back_super_zoom.jpg', 'https://product.koovs.com/170257_c00be7a2068d4d889e5b94fab24271c4_left_super_zoom.jpg', 'https://product.koovs.com/170257_c00be7a2068d4d889e5b94fab24271c4_right_super_zoom.jpg']
['L', 'XL']


In [51]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

## Tshirts

In [52]:
product_links={}
counter=0
for i in range(0,15):
    try:
        data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fmen%2Ft-shirts-and-polo-shirts&page-size=100&sort=relevance&page={i}',headers=headers)
        data_in_json=json.loads(data.text)
        for j in data_in_json['data'][0]['data']:
            product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
            counter+=1
    except:
        continue
len(product_links)

1322

In [53]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
category={}
mrp={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='T-Shirts'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break 
        except:
            continue
    #break
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

(1322, 1322, 1322, 1322, 1322, 1322, 1322, 1322, 1322)

In [54]:
value_tester(1000)

https://www.koovs.com/blue-saint-mens-tshirt-141357-158421.html
499
499
Made from cotton V-neckline Short sleeves Tipped and ribbed edges Slim fit
Men
Blue Saint
V-Neck Short Sleeves T-shirt
T-Shirts
https://product.koovs.com/158421_83c48a267db64b11be2aa0f2c8ab5803_image1_super_zoom.jpg
['https://product.koovs.com/158421_83c48a267db64b11be2aa0f2c8ab5803_front_super_zoom.jpg', 'https://product.koovs.com/158421_83c48a267db64b11be2aa0f2c8ab5803_back_super_zoom.jpg', 'https://product.koovs.com/158421_83c48a267db64b11be2aa0f2c8ab5803_left_super_zoom.jpg']
['S', 'M', 'L']


In [55]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

## Jeans

In [56]:
product_links={}
counter=0
for i in range(0,15):
    try:
        data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fmen%2Fjeans&page-size=100&sort=relevance&page={i}',headers=headers)
        data_in_json=json.loads(data.text)
        for j in data_in_json['data'][0]['data']:
            product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
            counter+=1
    except:
        continue
len(product_links)

431

In [57]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
category={}
mrp={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Jeans'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

(431, 431, 431, 431, 431, 431, 431, 431, 431)

In [58]:
value_tester(300)

https://www.koovs.com/blue-saint-men-skinny-jeans-148632-165845.html
2499
1749
Made from cotton-elastane blend Button and fly closure Classic five-pocket style Dark wash effect Chain detailing Belt loops Slim fit
Men
Blue Saint
Dark Wash Chain Detail Jeans
Jeans
https://product.koovs.com/165845_d1f6c547b31444a98963cfa7a9b0f8c8_image1_super_zoom.jpg
['https://product.koovs.com/165845_d1f6c547b31444a98963cfa7a9b0f8c8_front_super_zoom.jpg', 'https://product.koovs.com/165845_d1f6c547b31444a98963cfa7a9b0f8c8_back_super_zoom.jpg', 'https://product.koovs.com/165845_d1f6c547b31444a98963cfa7a9b0f8c8_left_super_zoom.jpg', 'https://product.koovs.com/165845_d1f6c547b31444a98963cfa7a9b0f8c8_right_super_zoom.jpg']
['30', '32', '34']


In [59]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

## Trousers

In [60]:
product_links={}
counter=0
for i in range(0,15):
    try:
        data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fmen%2Ftrousers-and-chinos&page-size=100&sort=relevance&page={i}',headers=headers)
        data_in_json=json.loads(data.text)
        for j in data_in_json['data'][0]['data']:
            product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
            counter+=1
    except:
        continue
len(product_links)

118

In [61]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
category={}
mrp={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Trousers'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

(118, 118, 118, 118, 118, 118, 118, 118, 118)

In [62]:
value_tester(100)

https://www.koovs.com/koovs-men-trousers-150030-167243.html
2699
2699
Made from cotton Button and fly closure Scenery print style Side pockets Belt loops Relaxed fit
Men
KOOVS
Placement Scenery Print Trousers
Trousers
https://product.koovs.com/167243_9414def4a2b345caa0cdb700439c0f19_image1_super_zoom.jpg
['https://product.koovs.com/167243_9414def4a2b345caa0cdb700439c0f19_front_super_zoom.jpg', 'https://product.koovs.com/167243_9414def4a2b345caa0cdb700439c0f19_back_super_zoom.jpg', 'https://product.koovs.com/167243_9414def4a2b345caa0cdb700439c0f19_left_super_zoom.jpg', 'https://product.koovs.com/167243_9414def4a2b345caa0cdb700439c0f19_right_super_zoom.jpg']
['30', '32', '34', '36']


In [63]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

## Jogger

In [64]:
product_links={}
counter=0
for i in range(0,15):
    try:
        data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fmen%2Fjog-pants&page-size=100&sort=relevance&page={i}',headers=headers)
        data_in_json=json.loads(data.text)
        for j in data_in_json['data'][0]['data']:
            product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
            counter+=1
    except:
        continue
len(product_links)

249

In [65]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
category={}
mrp={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Jogger'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

(249, 249, 249, 249, 249, 249, 249, 249, 249)

In [66]:
value_tester(200)

https://www.koovs.com/sheltr-mens-jogger-141090-158154.html
2299
1150
Made From Cotton and Elastane Two Pocket Style Zip Hem Closure Regular Fit
Men
Sheltr
Basic Zip Hem Jogger
Jogger
https://product.koovs.com/158154_d46d9f6b8c774fc19342bb44a6f1fdeb_image1_super_zoom.jpg
['https://product.koovs.com/158154_d46d9f6b8c774fc19342bb44a6f1fdeb_front_super_zoom.jpg', 'https://product.koovs.com/158154_d46d9f6b8c774fc19342bb44a6f1fdeb_back_super_zoom.jpg', 'https://product.koovs.com/158154_d46d9f6b8c774fc19342bb44a6f1fdeb_left_super_zoom.jpg', 'https://product.koovs.com/158154_d46d9f6b8c774fc19342bb44a6f1fdeb_right_super_zoom.jpg']
['28', '30', '32', '34', '36']


In [67]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

## Vests

In [68]:
product_links={}
counter=0
for i in range(0,15):
    try:
        data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fmen%2Fvests&page-size=100&sort=relevance&page={i}',headers=headers)
        data_in_json=json.loads(data.text)
        for j in data_in_json['data'][0]['data']:
            product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
            counter+=1
    except:
        continue
len(product_links)

93

In [69]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
category={}
mrp={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Vests'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

(93, 93, 93, 93, 93, 93, 93, 93, 93)

In [70]:
value_tester(80)

https://www.koovs.com/blue-saint-mens-vest-149678-166891.html
699
699
Made from cotton Round neck Back graphic print Sleeveless Oversized fit
Men
Blue Saint
Back Graphic Print Casual Vest
Vests
https://product.koovs.com/166891_b03ef77331d34c79ac98642cdb2fc117_image1_super_zoom.jpg
['https://product.koovs.com/166891_b03ef77331d34c79ac98642cdb2fc117_front_super_zoom.jpg', 'https://product.koovs.com/166891_b03ef77331d34c79ac98642cdb2fc117_back_super_zoom.jpg', 'https://product.koovs.com/166891_b03ef77331d34c79ac98642cdb2fc117_left_super_zoom.jpg', 'https://product.koovs.com/166891_b03ef77331d34c79ac98642cdb2fc117_right_super_zoom.jpg']
['S', 'M', 'L', 'XL']


In [71]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

## Shorts

In [72]:
product_links={}
counter=0
for i in range(0,15):
    try:
        data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fmen%2Fshorts&page-size=100&sort=relevance&page={i}',headers=headers)
        data_in_json=json.loads(data.text)
        for j in data_in_json['data'][0]['data']:
            product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
            counter+=1
    except:
        continue
len(product_links)

196

In [73]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
category={}
mrp={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Shorts'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

(196, 196, 196, 196, 196, 196, 196, 196, 196)

In [74]:
value_tester(100)

https://www.koovs.com/realm-mens-shorts-138942-155941.html
1799
1259
Made from cotton-spandex blend  Button and fly closure All over Print Three Pockets Slim fit
Men
REALM
Printed Casual Shorts
Shorts
https://product.koovs.com/155941_a5a847c3aa164e809dc115e1de5d3196_image1_super_zoom.jpg
['https://product.koovs.com/155941_a5a847c3aa164e809dc115e1de5d3196_front_super_zoom.jpg', 'https://product.koovs.com/155941_a5a847c3aa164e809dc115e1de5d3196_back_super_zoom.jpg', 'https://product.koovs.com/155941_a5a847c3aa164e809dc115e1de5d3196_left_super_zoom.jpg', 'https://product.koovs.com/155941_a5a847c3aa164e809dc115e1de5d3196_right_super_zoom.jpg']
['28', '30', '32', '34', '36']


In [75]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

## Hoodies

In [76]:
product_links={}
counter=0
for i in range(0,15):
    try:
        data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fmen%2Fhoodies-and-sweatshirts&page-size=100&sort=relevance&page={i}',headers=headers)
        data_in_json=json.loads(data.text)
        for j in data_in_json['data'][0]['data']:
            product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
            counter+=1
    except:
        continue
len(product_links)

121

In [77]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
category={}
mrp={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Hoodies'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

(121, 121, 121, 121, 121, 121, 121, 121, 121)

In [78]:
value_tester(100)

https://www.koovs.com/abg-men-tshirts-143344-160441.html
1499
1499
Made from cotton Crew neckline Back slogan print detail Long sleeves Regular fit
Men
ABG
Back Slogan Print Long Sleeve Sweatshirt
Hoodies
https://product.koovs.com/160441_a37b72a91daa496f876b657acd60e29f_image1_super_zoom.jpg
['https://product.koovs.com/160441_a37b72a91daa496f876b657acd60e29f_front_super_zoom.jpg', 'https://product.koovs.com/160441_a37b72a91daa496f876b657acd60e29f_back_super_zoom.jpg', 'https://product.koovs.com/160441_a37b72a91daa496f876b657acd60e29f_left_super_zoom.jpg']
['S', 'M', 'L']


In [79]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

## Jackets

In [80]:
product_links={}
counter=0
for i in range(0,15):
    try:
        data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fmen%2Fcoats-and-jackets&page-size=100&sort=relevance&page={i}',headers=headers)
        data_in_json=json.loads(data.text)
        for j in data_in_json['data'][0]['data']:
            product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
            counter+=1
    except:
        continue
len(product_links)

252

In [81]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
category={}
mrp={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Jackets'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

(252, 252, 252, 252, 252, 252, 252, 252, 252)

In [82]:
value_tester(200)

https://www.koovs.com/nylon-reflective-cut-and-sew-windcheater-jacket-151815-169037.html
1999
1099
Made from cotton Zip-up closure Cut & Sew style Full sleeves Slim fit
Men
FUGAZEE
Reflective Cut & Sew Jacket
Jackets
https://product.koovs.com/169037_92cfd00a584c4b92a36b28c683a1d418_image1_super_zoom.jpg
['https://product.koovs.com/169037_92cfd00a584c4b92a36b28c683a1d418_front_super_zoom.jpg', 'https://product.koovs.com/169037_92cfd00a584c4b92a36b28c683a1d418_back_super_zoom.jpg', 'https://product.koovs.com/169037_92cfd00a584c4b92a36b28c683a1d418_left_super_zoom.jpg', 'https://product.koovs.com/169037_92cfd00a584c4b92a36b28c683a1d418_right_super_zoom.jpg']
['M']


In [83]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()

## Shoes

In [84]:
product_links={}
counter=0
for i in range(0,15):
    try:
        data=rt.get(f'https://www.koovs.com/search-service/v1/products/listing/complete?href=https%3A%2F%2Fwww.koovs.com%2Fmen%2Ffootwear&page-size=100&sort=relevance&page={i}',headers=headers)
        data_in_json=json.loads(data.text)
        for j in data_in_json['data'][0]['data']:
            product_links[counter]='https://www.koovs.com'+j['links'][0]['href']
            counter+=1
    except:
        continue
len(product_links)

99

In [85]:
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
category={}
mrp={}
description={}
gender={}
for i in range(len(product_links)):
    data=rt.get(product_links[i],headers=headers)
    soup=bs4.BeautifulSoup(data.text,'html.parser')
    temp=soup.find_all('script')
    for j in temp:
        try:
            json_data = json.loads(j.contents[0][j.contents[0].index('{'):])
            if json_data['productPage']['productData']:
                brand_names[i]=json_data['productPage']['productData']['brandName']
                product_names[i]=json_data['productPage']['productData']['productName']
                img_temp=json_data['productPage']['productData']['imageUrls']
                primary_image_links_dict[i]=img_temp[0]
                secondary_image_links_dict[i]=img_temp[1:]
                price[i]=json_data['productPage']['productData']['discountPrice']
                mrp[i]=json_data['productPage']['productData']['price']
                gender[i]=json_data['productPage']['productData']['gender']
                category[i]='Shoes'
                temp_sizes=[]
                for k in json_data['productPage']['productAttribute']['sizes']:
                    if k['isOutOfStock'] == False:
                        temp_sizes.append(k['code'])
                sizes[i]=temp_sizes
                soup2=bs4.BeautifulSoup(json_data['productPage']['productData']['productDescription'],'html.parser')
                temp_description=[]
                for z in soup2.find_all('li'):
                    temp_description.append(z.text)
                description[i]=' '.join(temp_description)
                #break
        except:
            continue
    #break
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

(99, 99, 99, 99, 99, 99, 99, 99, 99)

In [87]:
value_tester(88)

https://www.koovs.com/marcello--ferri-owl-embroidered-loafers-148612-165825.html
2499
1250
Made from PU Round toe Velvet finish Owl embroidered detail Slip-on style Slightly raised sole
Men
Marcello & Ferri
Owl Embroidered Loafers
Shoes
https://product.koovs.com/165825_347a909a884043998f35547ebb932059_image1_super_zoom.jpg
['https://product.koovs.com/165825_347a909a884043998f35547ebb932059_front_super_zoom.jpg', 'https://product.koovs.com/165825_347a909a884043998f35547ebb932059_back_super_zoom.jpg', 'https://product.koovs.com/165825_347a909a884043998f35547ebb932059_left_super_zoom.jpg']
['UK 6/US 7', 'UK 7/US 8', 'UK 8/US 9', 'UK 9/US 10', 'UK 10/US 11']


In [88]:
con=sq.connect(r"Koovs.db")  
cur=con.cursor()
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links_dict.keys() and i in category.keys():
            if brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and category[i] !='':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.koovs.com/',product_links[i],product_names[i],brand_names[i],category[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links_dict[i],','.join(secondary_image_links_dict[i]),''))
    except:
        pass
con.commit()
con.close()